In [4]:
pip install tweepy

     |████████████████████████████████| 147 kB 263 kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
from tweepy import OAuthHandler
from tweepy import API

# Consumer key authentication
auth = OAuthHandler("qIwnNNJMkjBzFGXbvIfDCx5L1", "qBDTDdhi0PtgPg2SBIdZGOE5KpXqcjjQHdMXIXazbfa6wkxpb2")

# Access key authentication
auth.set_access_token("1338511495739674629-uCRKkRhkSBdUJO45rzkn4am36WEM4w", "889fsak4U775JstxLLgfaUcrEVYoJJS7ikBUUQNaMl5m4")

# Set up the API with the authentication handler
api = API(auth)

In [2]:
from tweepy.streaming import StreamListener
import json
import time
import sys

class SListener(StreamListener):
    def __init__(self, api = None, fprefix = 'streamer'):
        self.api = api or API()
        self.counter = 0
        self.fprefix = fprefix
        self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')


    def on_data(self, data):
        if  'in_reply_to_status' in data:
            self.on_status(data)
        elif 'delete' in data:
            delete = json.loads(data)['delete']['status']
            if self.on_delete(delete['id'], delete['user_id']) is False:
                return False
        elif 'limit' in data:
            if self.on_limit(json.loads(data)['limit']['track']) is False:
                return False
        elif 'warning' in data:
            warning = json.loads(data)['warnings']
            print("WARNING: %s" % warning['message'])
            return


    def on_status(self, status):
        self.output.write(status)
        self.counter += 1
        if self.counter >= 20000:
            self.output.close()
            self.output  = open('%s_%s.json' % (self.fprefix, time.strftime('%Y%m%d-%H%M%S')), 'w')
            self.counter = 0
        return


    def on_delete(self, status_id, user_id):
        print("Delete notice")
        return


    def on_limit(self, track):
        print("WARNING: Limitation notice received, tweets missed: %d" % track)
        return


    def on_error(self, status_code):
        print('Encountered error with status code:', status_code)
        return 


    def on_timeout(self):
        print("Timeout, sleeping for 60 seconds...")
        time.sleep(60)
        return

In [3]:
from tweepy import Stream

# Set up words to track
keywords_to_track = ["welcome","today"]

# Instantiate the SListener object 
listen = SListener(api)

# Instantiate the Stream object
stream = Stream(auth, listen)

# Begin collecting data
stream.filter(track = keywords_to_track)

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))

In [4]:
 pip install simplejson

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install simplejson==3.16.0


     |████████████████████████████████| 81 kB 81 kB/s  eta 0:00:01
  Created wheel for simplejson: filename=simplejson-3.16.0-cp37-cp37m-linux_x86_64.whl size=124595 sha256=4453c653e45d33578731e1cf6809f821390f104cc2b178dce60ad4e6c5f7e967
  Stored in directory: /home/client/.cache/pip/wheels/00/6f/29/109ee3b85a9f9861d4294b05e4597363d5ad3c9dfaccc78787
Successfully built simplejson
  Attempting uninstall: simplejson
    Found existing installation: simplejson 3.17.2
    Uninstalling simplejson-3.17.2:
      Successfully uninstalled simplejson-3.17.2
Note: you may need to restart the kernel to use updated packages.


In [3]:
import glob
tweets = []
files  = list(glob.iglob('streamer_*.json'))
for f in files:
    fh = open(f, 'r', encoding = 'utf-8')
    tweets_json = fh.read().split("\n")

    ## remove empty lines
    tweets_json = list(filter(len, tweets_json))

    ## parse each tweet
    for tweet in tweets_json:
        try:
            tweet_obj = json.loads(tweet)

            ## flatten the file to include quoted status and retweeted status info
            if 'quoted_status' in tweet_obj:
                tweet_obj['quoted_status-text'] = tweet_obj['quoted_status']['text'] 
                tweet_obj['quoted_status-user-screen_name'] = tweet_obj['quoted_status']['user']['screen_name']

            if 'retweeted_status' in tweet_obj:
                tweet_obj['retweeted_status-user-screen_name'] = tweet_obj['retweeted_status']['user']['screen_name']
                tweet_obj['retweeted_status-text'] = tweet_obj['retweeted_status']['text']

            tweet_obj['user-screen_name'] = tweet_obj['user']['screen_name']

            tweets.append(tweet_obj)
        except:
            pass
print(len(tweets))

60512


In [4]:
## create pandas DataFrame for further analysis
import pandas as pd
df_tweet = pd.DataFrame(tweets)

In [5]:
pip install emoji

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install regex


Note: you may need to restart the kernel to use updated packages.


In [5]:
df_tweet.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,retweeted_status,retweeted_status-text,retweeted_status-user-screen_name,source,text,timestamp_ms,truncated,user,user-screen_name,withheld_in_countries
0,None,None,Tue Dec 15 13:21:24 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,{'created_at': 'Tue Dec 15 13:12:22 +0000 2020...,Guys Please help me to complete my 100 Followe...,imrohit34,"<a href=""http://twitter.com/download/android"" ...",RT @imrohit34: Guys Please help me to complete...,1608038484757,False,"{'id': 1324353424176029696, 'id_str': '1324353...",Anishachetri24,NaN
1,None,None,Tue Dec 15 13:21:24 +0000 2020,"[12, 61]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1338836551753740293, 'id_str...",NaN,0,False,low,...,NaN,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RobertAlai Dude this Khatib today. He was eve...,1608038484651,False,"{'id': 1125299044568444928, 'id_str': '1125299...",alienkos,NaN
2,None,None,Tue Dec 15 13:21:24 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",And today I need to stand strong again. It's o...,1608038484780,False,"{'id': 1155817040726249472, 'id_str': '1155817...",rollingnya,NaN
3,None,None,Tue Dec 15 13:21:24 +0000 2020,"[0, 37]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1338836556375842818, 'id_str...",NaN,0,False,low,...,NaN,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",Soon you're going to run out of fuel. https://...,1608038484715,False,"{'id': 464866496, 'id_str': '464866496', 'name...",vkmutai,NaN
4,None,None,Tue Dec 15 13:21:24 +0000 2020,NaN,"{'hashtags': [{'text': 'FridayLivestream', 'in...",NaN,NaN,0,False,low,...,NaN,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",REQUEST @dimash_official @MTV #FridayLivestrea...,1608038484755,False,"{'id': 1316612154955624448, 'id_str': '1316612...",ET6gmalKlhO5D9b,NaN


In [6]:
import re
import emoji
import nltk
nltk.download('words')
words = set(nltk.corpus.words.words())
def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9_:]+","",str(tweet)) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", tweet) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    return tweet
df_tweet['text_clean'] = df_tweet['text'].map(lambda x: cleaner(x))

[nltk_data] Downloading package words to /home/client/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [7]:
df_tweet.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,retweeted_status-text,retweeted_status-user-screen_name,source,text,timestamp_ms,truncated,user,user-screen_name,withheld_in_countries,text_clean
0,None,None,Tue Dec 15 13:21:24 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,Guys Please help me to complete my 100 Followe...,imrohit34,"<a href=""http://twitter.com/download/android"" ...",RT @imrohit34: Guys Please help me to complete...,1608038484757,False,"{'id': 1324353424176029696, 'id_str': '1324353...",Anishachetri24,NaN,Please help me to complete my 100 Today ? Foll...
1,None,None,Tue Dec 15 13:21:24 +0000 2020,"[12, 61]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1338836551753740293, 'id_str...",NaN,0,False,low,...,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@RobertAlai Dude this Khatib today. He was eve...,1608038484651,False,"{'id': 1125299044568444928, 'id_str': '1125299...",alienkos,NaN,Dude this Khatib today . He was even thinner t...
2,None,None,Tue Dec 15 13:21:24 +0000 2020,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",And today I need to stand strong again. It's o...,1608038484780,False,"{'id': 1155817040726249472, 'id_str': '1155817...",rollingnya,NaN,And today I need to stand strong again . It ' ...
3,None,None,Tue Dec 15 13:21:24 +0000 2020,"[0, 37]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1338836556375842818, 'id_str...",NaN,0,False,low,...,NaN,NaN,"<a href=""http://twitter.com/download/android"" ...",Soon you're going to run out of fuel. https://...,1608038484715,False,"{'id': 464866496, 'id_str': '464866496', 'name...",vkmutai,NaN,Soon you ' re going to run out of fuel .
4,None,None,Tue Dec 15 13:21:24 +0000 2020,NaN,"{'hashtags': [{'text': 'FridayLivestream', 'in...",NaN,NaN,0,False,low,...,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",REQUEST @dimash_official @MTV #FridayLivestrea...,1608038484755,False,"{'id': 1316612154955624448, 'id_str': '1316612...",ET6gmalKlhO5D9b,NaN,REQUEST
